In [ ]:
import csv

characters = []

with open("HP_characters.csv", "r", encoding="utf8") as sent_file:
    csv = csv.reader(sent_file, delimiter=",")
    for row in csv:
        name = row[0].replace(' ', '_')
        parentage = row[1]
        house = row[2]
        occupation = row[3]
        characters.append((name, parentage, house, occupation))

#characters

In [ ]:
# Try to get the pages:
import urllib.request
import json
from urllib.parse import quote

url = 'http://example.com/'
response = urllib.request.urlopen(url)
data = response.read()      # a `bytes` object
text = data.decode('utf-8')

misses = []
lengths = []
disambig = []

#baseurl = "https://en.wikipedia.org/w/api.php?"

#https://harrypotter.fandom.com/wiki/Special:Search?query=Dumbledore&scope=internal&navigationSearch=true&so=trending
#https://www.reddit.com/r/learnpython/comments/muwu7v/scraping_fandomwiki_pages/
baseurl = "https://harrypotter.fandom.com/api.php?"

action = "action=query"

for character in characters:
    title = "titles=" + character[0]
    #print(title)
    content = "prop=revisions&rvprop=content"
    dataformat ="format=json"
    query = "{}{}&{}&{}&{}".format(baseurl, action, content, title, dataformat)
    #print(query)
    #try:
    wikiresponse = urllib.request.urlopen(query)
    wikidata = wikiresponse.read()
    wikitext = wikidata.decode('utf-8')
    lengths.append((character, len(wikitext)))

    #print(wikitext)
    try:
        jsonobj = json.loads(wikitext)
    except ValueError: 
        print(f'Decoding JSON has failed for {character}, moving on...')
    num = list(jsonobj['query']['pages'].keys())[0]
    wikitext = jsonobj['query']['pages'][num]['revisions'][0]['*']
    #wikitext = jsonobj['query']['pages'][num]['revisions'][0]['slots']['main']['*']
    f_name = character[0] + '.txt'
    with open("characters/" + f_name, 'w') as f:
        f.write(wikitext)


    #print(len(wikitext))
      #except:
  #  misses.append(quote(character))

In [ ]:

print(sorted(lengths, key=lambda x: x[1]))

In [ ]:
import os

unzipped = list(list(zip(*characters))[0])
names = []

for file in os.listdir("characters"):
    names.append(file[:-4])

print(len(names))
print(len(unzipped))    
i = 1
for name in unzipped:
    if name in names:
        print(f"{i} {name}")
        i = i + 1
        
print(len(list(set(unzipped))))

In [ ]:
for n in unzipped:
    print(n)